In [1]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
import numpy as np
import inspect

In [ ]:
import matplotlib.colors as mcolors
from matplotlib import cbook

In [29]:
# fake data
np.random.seed(19680801)

N = 50
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
area = (30 * np.random.rand(N))**2    # 0 to 15 point radii

In [3]:
%matplotlib widget

In [4]:
fig, ax = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
x = np.ma.ravel(x)
y = np.ma.ravel(y)
s = np.ma.ravel(area)

In [18]:
if edgecolors is None and not rcParams['_internal.classic_mode']:
    edgecolors = rcParams['scatter.edgecolors']

In [33]:
kwcolor = None
c_was_none = c is None
c_is_mapped = False  # Unless proven otherwise below.
valid_shape = True  # Unless proven otherwise below.

In [35]:
c_is_string_or_strings = (
    isinstance(c, str)
    or (np.iterable(c) and len(c) > 0
        and isinstance(cbook.safe_first_element(c), str)))

In [39]:
c

array([0.78422978, 0.39704754, 0.15464683, 0.72119325, 0.8368309 ,
       0.9378552 , 0.57671382, 0.19502656, 0.11779397, 0.9506002 ,
       0.2836723 , 0.95656242, 0.90942546, 0.32842519, 0.95176557,
       0.11632148, 0.87431415, 0.98522547, 0.75033538, 0.98515647,
       0.99018024, 0.06097709, 0.00373598, 0.81250321, 0.58273672,
       0.15024598, 0.15217356, 0.5270595 , 0.4794052 , 0.09218994,
       0.51195743, 0.04283111, 0.80629615, 0.24219866, 0.8482111 ,
       0.69558986, 0.36132686, 0.50570144, 0.29417242, 0.66008457,
       0.53413535, 0.19481606, 0.86190568, 0.97802889, 0.88218663,
       0.52735917, 0.22140904, 0.20272247, 0.0755575 , 0.95256799])

In [41]:
c = np.asanyarray(c, dtype=float)

In [44]:
c = c.ravel()

In [45]:
c_is_mapped = True
colors = None

In [ ]:
    """
    Helper function to process color related arguments of `.Axes.scatter`.

    Argument precedence for facecolors:

    - c (if not None)
    - kwargs['facecolors']
    - kwargs['facecolor']
    - kwargs['color'] (==kwcolor)
    - 'b' if in classic mode else the result of ``get_next_color_func()``

    Argument precedence for edgecolors:

    - edgecolors (is an explicit kw argument in scatter())
    - kwargs['edgecolor']
    - kwargs['color'] (==kwcolor)
    - 'face' if not in classic mode else None

    Parameters
    ----------
    c : color or sequence or sequence of color or None
        See argument description of `.Axes.scatter`.
    edgecolors : color or sequence of color or {'face', 'none'} or None
        See argument description of `.Axes.scatter`.
    kwargs : dict
        Additional kwargs. If these keys exist, we pop and process them:
        'facecolors', 'facecolor', 'edgecolor', 'color'
        Note: The dict is modified by this function.
    xsize : int
        The size of the x and y arrays passed to `.Axes.scatter`.
    get_next_color_func : callable
        A callable that returns a color. This color is used as facecolor
        if no other color is provided.

        Note, that this is a function rather than a fixed color value to
        support conditional evaluation of the next color.  As of the
        current implementation obtaining the next color from the
        property cycle advances the cycle. This must only happen if we
        actually use the color, which will only be decided within this
        method.

    Returns
    -------
    c
        The input *c* if it was not *None*, else a color derived from the
        other inputs or defaults.
    colors : array(N, 4) or None
        The facecolors as RGBA values, or *None* if a colormap is used.
    edgecolors
        The edgecolor.

    """

In [11]:
def _parse_scatter_color_args(c, edgecolors, kwargs, xsize,
                              get_next_color_func):

    facecolors = kwargs.pop('facecolors', None)
    facecolors = kwargs.pop('facecolor', facecolors)
    edgecolors = kwargs.pop('edgecolor', edgecolors)

    kwcolor = kwargs.pop('color', None)

    if kwcolor is not None and c is not None:
        raise ValueError("Supply a 'c' argument or a 'color'"
                         " kwarg but not both; they differ but"
                         " their functionalities overlap.")

    if kwcolor is not None:
        try:
            mcolors.to_rgba_array(kwcolor)
        except ValueError as err:
            raise ValueError(
                "'color' kwarg must be an color or sequence of color "
                "specs.  For a sequence of values to be color-mapped, use "
                "the 'c' argument instead.") from err
        if edgecolors is None:
            edgecolors = kwcolor
        if facecolors is None:
            facecolors = kwcolor

    if edgecolors is None and not rcParams['_internal.classic_mode']:
        edgecolors = rcParams['scatter.edgecolors']

    c_was_none = c is None
    if c is None:
        c = (facecolors if facecolors is not None
             else "b" if rcParams['_internal.classic_mode']
             else get_next_color_func())
    c_is_string_or_strings = (
        isinstance(c, str)
        or (np.iterable(c) and len(c) > 0
            and isinstance(cbook.safe_first_element(c), str)))

    def invalid_shape_exception(csize, xsize):
        return ValueError(
            f"'c' argument has {csize} elements, which is inconsistent "
            f"with 'x' and 'y' with size {xsize}.")

    c_is_mapped = False  # Unless proven otherwise below.
    valid_shape = True  # Unless proven otherwise below.
    if not c_was_none and kwcolor is None and not c_is_string_or_strings:
        try:  # First, does 'c' look suitable for value-mapping?
            c = np.asanyarray(c, dtype=float)
        except ValueError:
            pass  # Failed to convert to float array; must be color specs.
        else:
            # handle the documented special case of a 2D array with 1
            # row which as RGB(A) to broadcast.
            if c.shape == (1, 4) or c.shape == (1, 3):
                c_is_mapped = False
                if c.size != xsize:
                    valid_shape = False
            # If c can be either mapped values or a RGB(A) color, prefer
            # the former if shapes match, the latter otherwise.
            elif c.size == xsize:
                c = c.ravel()
                c_is_mapped = True
            else:  # Wrong size; it must not be intended for mapping.
                if c.shape in ((3,), (4,)):
                    _log.warning(
                        "*c* argument looks like a single numeric RGB or "
                        "RGBA sequence, which should be avoided as value-"
                        "mapping will have precedence in case its length "
                        "matches with *x* & *y*.  Please use the *color* "
                        "keyword-argument or provide a 2-D array "
                        "with a single row if you intend to specify "
                        "the same RGB or RGBA value for all points.")
                valid_shape = False
    if not c_is_mapped:
        try:  # Is 'c' acceptable as PathCollection facecolors?
            colors = mcolors.to_rgba_array(c)
        except (TypeError, ValueError) as err:
            if "RGBA values should be within 0-1 range" in str(err):
                raise
            else:
                if not valid_shape:
                    raise invalid_shape_exception(c.size, xsize) from err
                # Both the mapping *and* the RGBA conversion failed: pretty
                # severe failure => one may appreciate a verbose feedback.
                raise ValueError(
                    f"'c' argument must be a color, a sequence of colors, "
                    f"or a sequence of numbers, not {c}") from err
        else:
            if len(colors) not in (0, 1, xsize):
                # NB: remember that a single color is also acceptable.
                # Besides *colors* will be an empty array if c == 'none'.
                raise invalid_shape_exception(len(colors), xsize)
    else:
        colors = None  # use cmap, norm after collection is created
    return c, colors, edgecolors

In [46]:
c = np.ma.masked_invalid(c)

In [52]:
linewidths = None

In [53]:
x, y, s, edgecolors, linewidths = _combine_masks(x, y, s, edgecolors, linewidths)

In [48]:
def is_scalar_or_string(val):
    """Return whether the given object is a scalar or string like."""
    return isinstance(val, str) or not np.iterable(val)

In [49]:
def safe_masked_invalid(x, copy=False):
    x = np.array(x, subok=True, copy=copy)
    if not x.dtype.isnative:
        # Note that the argument to `byteswap` is 'inplace',
        # thus if we have already made a copy, do the byteswap in
        # place, else make a copy with the byte order swapped.
        # Be explicit that we are swapping the byte order of the dtype
        x = x.byteswap(copy).newbyteorder('S')

    try:
        xm = np.ma.masked_invalid(x, copy=False)
        xm.shrink_mask()
    except TypeError:
        return x
    return xm

In [50]:
def _combine_masks(*args):
    """
    Find all masked and/or non-finite points in a set of arguments,
    and return the arguments as masked arrays with a common mask.

    Arguments can be in any of 5 categories:

    1) 1-D masked arrays
    2) 1-D ndarrays
    3) ndarrays with more than one dimension
    4) other non-string iterables
    5) anything else

    The first argument must be in one of the first four categories;
    any argument with a length differing from that of the first
    argument (and hence anything in category 5) then will be
    passed through unchanged.

    Masks are obtained from all arguments of the correct length
    in categories 1, 2, and 4; a point is bad if masked in a masked
    array or if it is a nan or inf.  No attempt is made to
    extract a mask from categories 2 and 4 if :meth:`np.isfinite`
    does not yield a Boolean array.  Category 3 is included to
    support RGB or RGBA ndarrays, which are assumed to have only
    valid values and which are passed through unchanged.

    All input arguments that are not passed unchanged are returned
    as masked arrays if any masked points are found, otherwise as
    ndarrays.

    """
    if not len(args):
        return ()
    if is_scalar_or_string(args[0]):
        raise ValueError("First argument must be a sequence")
    nrecs = len(args[0])
    margs = []  # Output args; some may be modified.
    seqlist = [False] * len(args)  # Flags: True if output will be masked.
    masks = []    # List of masks.
    for i, x in enumerate(args):
        if is_scalar_or_string(x) or len(x) != nrecs:
            margs.append(x)  # Leave it unmodified.
        else:
            if isinstance(x, np.ma.MaskedArray) and x.ndim > 1:
                raise ValueError("Masked arrays must be 1-D")
            x = np.asanyarray(x)
            if x.ndim == 1:
                x = safe_masked_invalid(x)
                seqlist[i] = True
                if np.ma.is_masked(x):
                    masks.append(np.ma.getmaskarray(x))
            margs.append(x)  # Possibly modified.
    if len(masks):
        mask = np.logical_or.reduce(masks)
        for i, x in enumerate(margs):
            if seqlist[i]:
                margs[i] = np.ma.array(x, mask=mask)
    return margs